# FIRST TRY

ora proviamo, con tutto ciò che fino ad ora abbiamo implementato, a sviluppare qualcosa.

plausibile che useremo come primo esperimento niente meno che "jump", il gioco del dinosauro di google.

In [1]:
import numpy as np
import keras
import random
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


usiamo (copia ed incollando) la rescaled probability function

In [2]:
def target_linear_rescaled_distribution(reward, royal_action, starting_distribution):

    starting_distribution = np.array(starting_distribution)

    if reward > 1:
        reward = 1

    assert reward >= -1, 'reward must be between -1 and 1'

    if starting_distribution[royal_action] == 1 and reward >= 0:
        return starting_distribution
    elif starting_distribution[royal_action] == 1 and reward < 0: # in this case we want to uniformize the others
        new_royal_probability = min(1, starting_distribution[royal_action]*(1 + reward))
        rest = 1 - new_royal_probability
        n = len(starting_distribution)
        new_distribution = np.ones(n)*rest/(n-1)
        new_distribution[royal_action] = new_royal_probability
        return new_distribution
    
    coefficient = np.zeros(len(starting_distribution))
    new_royal_probability = min(1, starting_distribution[royal_action]*(1 + reward))
    coefficient[royal_action] = new_royal_probability # this is not the real coefficient we use it to be quicker

    if new_royal_probability == 1:
        return coefficient #just a trick to be quicker in case we hit a 1
    
    rest = starting_distribution[royal_action]*reward 
    sum_others = 1 - starting_distribution[royal_action]
    ratio = rest/sum_others

    coefficient = np.ones(len(starting_distribution))*(1 - ratio)
    coefficient[royal_action] = (1 + reward)
    
    return starting_distribution*coefficient.T

la custom network (anch'essa copia ed incollata)

In [3]:
class CustomModel(keras.Model):
    def __init__(self, input_size, number_of_hidden, output_size, hidden_size, activ = "relu", out_activ = "softmax"):
        super().__init__()
        self.input1 = keras.Input(shape=(input_size,)) # gli input sono 4
        self.hidden = []
        for i in range (number_of_hidden):
            self.hidden.append(keras.layers.Dense(hidden_size[i], activation = activ))
        self.output1 = keras.layers.Dense(output_size, activation = out_activ)

    def call(self, inputs):
        x = inputs
        for h in self.hidden:
            x = h(x)
        return self.output1(x)

ed ora definiamo il gioco

In [11]:
# Inizializzazione di pygame
pygame.init()

# Dimensioni della finestra
width, height = 800, 500
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("player Jump")

# Colori
white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)

# Parametri del player
player_width, player_height = 50, 50
player_x = 50
player_y = height - player_height - 20
player_velocity = 20
gravity = 1
jump_strength = -20

ground = height - 20

obstacle_width, obstacle_height = 20, 40
obstacle_speed = 10
obstacle_frequency = 1500  # millisecondi tra l'apparizione di un nuovo ostacolo

# Inizializza il clock
clock = pygame.time.Clock()

# Lista degli ostacoli
obstacles = []
spawn_time = 0

# Variabili del player
is_jumping = False
player_y_velocity = 0

# Costruzione del player
player = CustomModel(input_size=10, number_of_hidden = 2, output_size = 2, hidden_size = [12, 6])
player.compile(optimizer=keras.optimizers.Adam(), loss= keras.losses.CategoricalCrossentropy() , metrics=[keras.metrics.CategoricalAccuracy()])
env = [player_width, player_y, player_velocity, player_y_velocity, ground, obstacle_width, obstacle_height, 800, obstacle_speed, int(is_jumping)]
# in env ci sono : player_width, player_y, player_velocity, jump_strength, ground, obstacle_width_ obstacle_height, first_object_x, obstacle_speed

# Funzione per disegnare il playersauro
def draw_player(x, y):
    pygame.draw.rect(win, black, (x, y, player_width, player_height))

# Funzione per disegnare gli ostacoli
def draw_obstacle(obstacle):
    pygame.draw.rect(win, red, obstacle)

# Funzione per controllare le collisioni
def check_collision(player_rect, obstacles):
    for obstacle in obstacles:
        if player_rect.colliderect(obstacle):
            return True
    return False

# Funzione per mostrare il punteggio
def draw_score(win, score, font_size=36):
    font = pygame.font.SysFont(None, font_size)
    score_text = font.render(f"Score: {score}", True, (0, 0, 0))
    text_rect = score_text.get_rect(center=(width // 2, 20))
    win.blit(score_text, text_rect)

def draw_gen(win, gen, font_size=36):
    font = pygame.font.SysFont(None, font_size)
    score_text = font.render(f"Gen: {gen}", True, (0, 0, 0))
    text_rect = score_text.get_rect(center=(width*0.75 , 20))
    win.blit(score_text, text_rect)

fps = 60
update = 6
goal = 10 * update

# Learning parameters
time_window = 6 # è il numero di passi dalla morte che definiamo "problematici"
deep = 10

gen = 100

for g in range (gen):

    # reset variabili
     
    # player memory 
    history = []
    royal_history = []
    reward_history = []
    env_history = []

    # Lista degli ostacoli
    obstacles = []
    spawn_time = 0
    obstacle_speed = 10
    obstacle_frequency = 1500  # millisecondi tra l'apparizione di un nuovo ostacolo

    # Variabili del player
    is_jumping = False
    player_y_velocity = 0
    player_x = 50
    player_y = height - player_height - 20

    # game info
    score = 0
    time = 0
    conta_frame = 0
    keys = 0
    global_index = 0

    last_jump = 0

    # Loop principale
    running = True

    while running:
        clock.tick(fps)
        win.fill(white)
        
        conta_frame += 1           

        if conta_frame == fps//update:

            time += 1/6
            conta_frame = 0
            global_index += 1

            if len(obstacles) > 0:
                env = [player_width, player_y, player_velocity, player_y_velocity, ground, obstacle_width, obstacle_height, obstacles[0].x, obstacle_speed, int(is_jumping)]
            else:
                env = [player_width, player_y, player_velocity, player_y_velocity, ground, obstacle_width, obstacle_height, 800, obstacle_speed, int(is_jumping)]
            env_history.append(env)
            space = player(np.array([env])) # queste sono le probabilità di saltare e non saltare
            keys = 1 if random.uniform(0, 1) > space[0][0] else 0
            history.append(space[0])
            royal_history.append(keys)
            rew = 1 # default -> sta vivendo quindi va bene!
            reward_history.append(rew)
            if keys == 1 and is_jumping:
                reward_history[global_index - 1] = -1
            
        draw_score(win, score, font_size=36)
        draw_gen(win, g, font_size=36)

        # Gestione eventi
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                running = False

        # Gestione salto
        if keys == 1 and not is_jumping:
            is_jumping = True
            last_jump = global_index - 1 # questo è il momento dell'ultimo salto prima di morire!
            n = min(last_jump, time_window)
            for i in range (n):
                reward_history[n-i-1] = -(n-i)/(n-i+1) # in generale non vogliamo che salti a caso, se poi questo salto andrà bene è un altro discorso
            player_y_velocity = jump_strength

        # Movimento del player
        if is_jumping:
            player_y += player_y_velocity
            player_y_velocity += gravity
            if player_y >= ground - player_height:
                player_y = ground - player_height
                is_jumping = False

        # Gestione degli ostacoli
        current_time = pygame.time.get_ticks()
        if current_time - spawn_time > obstacle_frequency:
            obstacle_x = width
            obstacle_y = ground - obstacle_height
            obstacles.append(pygame.Rect(obstacle_x, obstacle_y, obstacle_width, obstacle_height))
            spawn_time = current_time
            obstacle_frequency = random.randint(800, 1500)
            obstacle_speed += 0.1

        # Movimento degli ostacoli
        for obstacle in obstacles:
            obstacle.x -= obstacle_speed
            draw_obstacle(obstacle)

        # Rimozione degli ostacoli fuori schermo
        old_len = len(obstacles)
        obstacles = [obstacle for obstacle in obstacles if obstacle.x + obstacle_width > 0]

        # Premiamo i salti che ci fanno vivere
        if old_len > len(obstacles): 
            score += 1
            n = min(last_jump, time_window)
            for i in range (n):
                reward_history[n-i-1] = (n-i+1)/(n-i+1) # se è un buon salto lo premiamo

        # Controllo delle collisioni
        player_rect = pygame.Rect(player_x, player_y, player_width, player_height)
        if check_collision(player_rect, obstacles):
            print(f"Hai perso! Score : {score}")
            steps = len(royal_history)
            x = env_history

            # Puniamo i comportamenti che ci hanno portato a morire
            if last_jump != 0 and is_jumping:
                print("JUMP-DEATH")
                n = min(time_window, last_jump)
                for i in range (n):
                    reward_history[last_jump - i] =  -1 # Non doveva fare l'ultimo salto in quel momento
                x = x[:last_jump+1] # Togliamo l'ultima parte di memoria (poiché tanto è già morto)
                reward_history = reward_history[:last_jump+1]
                royal_history = royal_history[:last_jump+1]
                history = history[:last_jump+1]
                steps = len(x)
            else:
                print("NON-JUMP-DEATH")
                n = min(time_window, steps)
                for i in range (n):
                    reward_history[steps -i - 1] = -1 # Ha preso decisioni sbagliate negli ultimi attimi di vita
            
            y = [target_linear_rescaled_distribution(reward_history[i], royal_history[i], history[i]) for i in range(steps)]
            player.fit(np.array([x]), np.array([y]), batch_size=steps, epochs=deep, verbose=0)      
            running = False    
        
        if score >= goal:
            print("Hai vinto!")
            steps = len(env_history)
            x = env_history
            y = [target_linear_rescaled_distribution(reward_history[i], royal_history[i], history[i]) for i in range(steps)]
            player.fit(np.array([x]), np.array([y]), batch_size=steps, epochs=deep, verbose=0)      
            running = False    

        # Disegna il player
        draw_player(player_x, player_y)

        # Aggiorna la finestra
        pygame.display.update()

pygame.quit()

Hai perso! Score : 0
[[50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 800, 10, 0], [50, 430, 20, 0, 480, 20, 40, 770, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 670, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 570, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 470, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 370, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 270, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 170, 10.1, 0]]
NON-JUMP-DEATH
15
Hai perso! Score : 0
[[50, 430, 20, 0, 480, 20, 40, 710, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 610, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 510, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 410, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 310, 10.1, 0], [50, 430, 20, 0, 480, 20, 40, 210, 10.2, 0], [50, 430,

error: display Surface quit

Da provare una specie di "maschera" sugli stati che se entriamo in uno stato già visto non lo studiamo due volte.

Funziona! E funziona anche molto bene (bisogna pur dire che il compito che deve risolvere in questo caso è molto semplice, ma comunque ci rende molto felici!)

Bisogna sistemare nuovamente la funzione che riscala il target, per il resto bene!